In [2]:
!pip install pycryptodome==3.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycryptodome: filename=pycryptodome-3.7.1-cp310-cp310-linux_x86_64.whl size=7511836 sha256=babff5415751d44a758c97a921c8618271bf0dd53402a3621608db219dc42800
  Stored in directory: /root/.cache/pip/wheels/5e/f5/12/2a4ad4d3448eeda90babdf70c11c4e318dbb9b3717d22a18f6
Successfully built pycryptodome


In [3]:
!pip install setuptools

In [4]:
!pip install cryptography

In [22]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.modes import XTS
from cryptography.hazmat.primitives import padding
import os


class AESCipher:
    def __init__(self, key_size=256, mode=AES.MODE_ECB):
        self.key_size = key_size
        self.mode = mode
        self.block_size = AES.block_size
        self.key = None

    def generate_key(self):
        self.key = get_random_bytes(self.key_size//8)

    def encrypt(self, data):
        if self.mode == AES.MODE_ECB:
            cipher = AES.new(self.key, self.mode)
            ct_bytes = cipher.encrypt(pad(data, self.block_size))
            return ct_bytes
        elif self.mode == AES.MODE_CBC:
            # Para o modo CBC, é necessário gerar um vetor de inicialização aleatório
            iv = get_random_bytes(self.block_size)
            cipher = AES.new(self.key, self.mode, iv)
            ct_bytes = cipher.encrypt(pad(data, self.block_size))
            return iv + ct_bytes
        elif self.mode == AES.MODE_CFB or self.mode == AES.MODE_OFB:
            # Para os modos CFB e OFB, o vetor de inicialização é gerado aleatoriamente
            iv = get_random_bytes(self.block_size)
            cipher = AES.new(self.key, self.mode, iv)
            ct_bytes = cipher.encrypt(data)
            return iv + ct_bytes
        elif self.mode == AES.MODE_CTR:
            # Para o modo CTR, o nonce é gerado aleatoriamente
            nonce = get_random_bytes(self.block_size//2)
            cipher = AES.new(self.key, self.mode, nonce=nonce)
            ct_bytes = cipher.encrypt(data)
            return nonce + ct_bytes


    def decrypt(self, ct_bytes):
        if self.mode == AES.MODE_ECB:
            cipher = AES.new(self.key, self.mode)
            pt = unpad(cipher.decrypt(ct_bytes), self.block_size)
            return pt
        elif self.mode == AES.MODE_CBC:
            # Para o modo CBC, o vetor de inicialização é os primeiros blocos do texto cifrado
            iv = ct_bytes[:self.block_size]
            ct_bytes = ct_bytes[self.block_size:]
            cipher = AES.new(self.key, self.mode, iv)
            pt = unpad(cipher.decrypt(ct_bytes), self.block_size)
            return pt
        elif self.mode == AES.MODE_CFB or self.mode == AES.MODE_OFB:
            # Para os modos CFB e OFB, o vetor de inicialização é os primeiros blocos do texto cifrado
            iv = ct_bytes[:self.block_size]
            ct_bytes = ct_bytes[self.block_size:]
            cipher = AES.new(self.key, self.mode, iv)
            pt = cipher.decrypt(ct_bytes)
            return pt
        elif self.mode == AES.MODE_CTR:
            # Para o modo CTR, o nonce é os primeiros blocos do texto cifrado
            nonce = ct_bytes[:self.block_size//2]
            ct_bytes = ct_bytes[self.block_size//2:]
            cipher = AES.new(self.key, self.mode, nonce=nonce)
            pt = cipher.decrypt(ct_bytes)
            return pt



# Instanciando a classe AESCipher com o modo CBC e tamanho da chave de 256 bits (32 bytes) ou 128 bits (16 bytes)
cipher = AESCipher(key_size=256, mode=AES.MODE_CTR)

# Gerando uma chave aleatória
cipher.generate_key()

# Mensagem a ser criptografada
message = u"Seguranca Computacional Criptografia Simetrica AES".encode('utf-8')

# Criptografando a mensagem
ciphertext = cipher.encrypt(message)

# Imprimindo o texto cifrado e a chave usada na criptografia
print("Texto cifrado: ", ciphertext)
print("Chave: ", cipher.key)

# Descriptografando a mensagem
decrypted_message = cipher.decrypt(ciphertext)

# Imprimindo o texto descriptografado
print("Texto descriptografado: ", decrypted_message)

Texto cifrado:  b'\xc9a\xa0\xbe\xe8C^&\xe8\x96K\xf6\xb8\xdd\x96\xf3\x91\x99\x0c\xc4\xb6"\xc3h\x9c\xd3\x0e\x02\xa2J;4KV\x9f\xa8\xfc\xce\xecp\xa8\x96[\xdc\xca)e\xfa9\xb15C5\xb4\xe8\xc6\x97E'
Chave:  b'4p\xf7|\x89\xa1\x10b\xd8\x81\xde{\xa5~\xc5\xc3z\x1by\n\xf3\x0b\xae\xfe\xf7\x13\x95n@\x1eA\xf0'
Texto descriptografado:  b'Seguranca Computacional Criptografia Simetrica AES'
